# Exploratory Data Analysis on a single SC2 Replay File

This notebook details the process for loading and exploring a single SC2 replay file. The following goals are achieved:
1. Load the replay file using `sc2reader`
2. Extract the events stored in the `.events` attribute of the replay file into a pandas dataframe.
3. Extract other pertinent information from the replay file such as: 
    * Filehash.
    * Datetime the game was played.
    * The map on which the game was played.
    * Length of the game.
    * Races of the players.
    * Skill levels of the players (by league).
    * The winner of the game.
4. Drop all unnecessary columns, convert the rest of the columns into dummy variables which indicate the category of the action and 1 if the action was taken.
5. Divide the data up into time chunks (10 seconds initially, to be revised later). Sum the number of 1s in each chunk to get the number of actions taken in each chunk.
6. Add columns to the dataframe indicating the number of actions taken in all previous chunks.
7. Add columns to the dataframe indicating player races, skill levels, and game winner.
8. Attempt a logistic regression using all chunks as inputs.
9. Investigate an optimization method which results in the earliest prediction that does not change until the end of the game. I.e., how early can the regression make a prediction which remains constant until the end of the game.

See __[this link](https://web.archive.org/web/20201031184319/https://miguelgondu.github.io/python/ai/video%20games/2018/09/04/a-tutorial-on-sc2reader-events-and-units.html)__ for the tutorial that was used in establishing how to work with, and extract data from an SC2 replay file, using `sc2reader`, and __[this link](https://github.com/IBM/starcraft2-replay-analysis)__ for an example of analyzing a single SC2 replay file using `sc2reader` and Jupyter Notebooks.

In [1]:
# imports and settings

import pandas as pd
import numpy as np
import sc2reader
from multiprocessing import Pool # for parallel processing of replay files
from scripts.classes import ReplayInfo
import re

## Load a Replay
We will use the `load_level=4` setting to extract the most detailed data possible from the replay file. This includes all events, units, and upgrades.

The replay selected is a professional replay between Serral (Zerg) and Showtime (Protoss). The replay is 42 minutes long (which is long for a StarCraft II game) which should be a good indication of the upper end of the processing time and requirements for a replay.

In [2]:
# Load a replay
replay_path = "C:/Users/jared/Gits/sc2-modeling/data/SpawningTool/Other/page1/2-liberators-1-tank-2.SC2Replay"
replay = sc2reader.load_replay(replay_path, load_level=4)

if len(replay.players) == 2:
    # Extract a list of replay events
    events_list = replay.raw_data['replay.game.events']
else:
    print("NOTE - This replay has more than 2 players. Not valid!!!")

In [3]:
replay_info = ReplayInfo(replay)
replay.raw_data['re']

In [20]:
RACE_LIST = [
            'Protoss',
            'Terran',
            'Zerg'
        ]

player = str(replay.players[0])

for race in RACE_LIST:

    race_string = '('+race+')'

    if race_string.lower() in player.lower():

        # delete race from the player1 string

        # assert that race_string is in player
        assert race_string in player, \
            f'{player} does not adhere to to {race_string} formatting'
        # use replace to delete the player race
        player = player.replace(race_string, '')

        # create regex to find 'Player 1 - ' leaving only actual name
        reg_str = r'Player\s\d\s\-\s'

        # assert that reg_str is in player string
        assert re.search(reg_str, player), \
            f'{player} does not adhere to to {reg_str} formatting'

        print(race)

# race_string = '(Terran)'
# race_string in player


Terran


## Extract Event Data from Replay
The following code extracts the events from the replay file and stores them in a pandas dataframe. Multiproccessing is used to speed up the process, as the replay file contains over 150,000 events.

Steps:
1. `events_list` now contains a list of stored event objects. `get_event_data` is a function that takes a specific event, and returns a dictionary of the values for each attribute of that event object. 
2. A multiprocessing pool iterates through all events returning a dictionary for each.
3. The output from the pool is stored in a list of dictionaries which can then be compressed into a pandas dataframe. A list of dictionaries is used so that all column names for all events do not need to be created manually.

In [17]:
# create function to get the event data so that multiprocessing can be used
def get_event_data(event):
    """
    get_event_data
    Extracts the data from the event and returns it as a dictionary
    Ignores events that start with '_', i.e., special attributes and dunder types

    Args:
        event (sc2reader.event): Event object extracted from sc2reader.events

    Returns:
        [dict]: A dictionary containing the event data
    """    
    # ignore attributes that are not needed (special or dunder)
    event_attributes = [attr for attr in dir(event) if not attr.startswith('_')]

    # initialize a dictionary to store the values of each attribute
    d = dict()

    # loop through each attribute and store the value in the dictionary
    for attr in event_attributes:
        # ignore attributes if they do not contain a value type
        if type(getattr(event, attr)) in [int, float, str, bool]:
            d[attr] = getattr(event, attr)
    
    return d

In [18]:
# run the function on each event in the list, returns a list of dictionaries
# with Pool() as pool:
#     dict_list = pool.map(get_event_data, events_list)
dict_list = [get_event_data(event) for event in events_list]
# convert the list of dictionaries to a dataframe
event_df = pd.DataFrame(dict_list)
event_df.shape

(13871, 35)

'UserOptionsEvent'

We not have an `event_df` with 151,759 rows and 112 columns.

## Processing the Dataframe
The major steps to accomplish here are:
1. Identify all columns.
2. Drop all columns that are not needed. We will also store a list of columns that are left after dropping in a csv file in the info directory. This list can be used later as a master list of columns for processing of all dataframes.
3. Check if columns are continuous or categorical (discrete integers are considered categorical).
4. Convert all categorical columns into dummy variables and drop the original column, and the first dummy column (using the pandas `drop first` option).
5. Remove all rows with non-player PID values (the players are the first two PIDs in the replay, all other PIDs are observers).
6. Summarise the dataframe by dividing it up into chunks of 10 seconds, summing up the count of all events during that time.


In [5]:
# # print the names of the columns to a csv
# pd.DataFrame(event_df.columns).to_csv(
#     '../info/raw_columns_list.csv',
#     index=False)

In [6]:
# read the csv into columns_checklist
columns_checklist = pd.read_csv('../info/raw_columns_list.csv')
columns_checklist

,column_name,include_bool,categorical_bool,agg_type
0,frame,1,0,none
1,name,1,1,sum
2,pid,1,1,none
3,second,1,0,diff
4,sid,0,0,none
...,...,...,...,...
107,target,0,0,none
108,text,0,0,none
109,to_all,0,0,none
110,to_allies,0,0,none


In [7]:
# create a dataframe of all columns with to be included
column_keep_df = columns_checklist.loc[
    columns_checklist['include_bool'] == 1].drop(['include_bool'], axis=1)
column_keep_df

,column_name,categorical_bool,agg_type
0,frame,0,none
1,name,1,sum
2,pid,1,none
3,second,0,diff
7,count,0,sum
8,upgrade_type_name,1,sum
13,unit_type_name,1,sum
26,ff_minerals_lost_army,0,sum
27,ff_minerals_lost_economy,0,sum
28,ff_minerals_lost_technology,0,sum


In [8]:
# We now have a list of 68 columns that we want to keep
# We can use this list to create a new dataframe with only the columns we want
event_df = event_df[column_keep_df['column_name']]
event_df.shape

KeyError: "['count', 'upgrade_type_name', 'unit_type_name', 'ff_minerals_lost_army', 'ff_minerals_lost_economy', 'ff_minerals_lost_technology', 'ff_vespene_lost_army', 'ff_vespene_lost_economy', 'ff_vespene_lost_technology', 'food_made', 'food_used', 'minerals_collection_rate', 'minerals_current', 'minerals_killed', 'minerals_killed_army', 'minerals_killed_economy', 'minerals_killed_technology', 'minerals_lost', 'minerals_lost_army', 'minerals_lost_economy', 'minerals_lost_technology', 'minerals_used_active_forces', 'minerals_used_current', 'minerals_used_current_army', 'minerals_used_current_economy', 'minerals_used_current_technology', 'minerals_used_in_progress', 'minerals_used_in_progress_army', 'minerals_used_in_progress_economy', 'minerals_used_in_progress_technology', 'resources_killed', 'resources_lost', 'resources_used_current', 'resources_used_in_progress', 'vespene_collection_rate', 'vespene_current', 'vespene_killed', 'vespene_killed_army', 'vespene_killed_economy', 'vespen

### Create Masks for Categorical and Continuous Columns
We will create two masks one for all categorical columns and one for all continuous columns.

First we need to identify which columns are continuous and which columns are categorical.

In [9]:
# identify which columns are continuous and which are categorical
categorical_columns = column_keep_df.loc[
        column_keep_df['categorical_bool'] == 1, 'column_name'].tolist()
continuous_columns = column_keep_df.loc[
        column_keep_df['categorical_bool'] == 0, 'column_name'].tolist()

In [10]:
print('<<<<Categorical columns>>>>\n',categorical_columns)

<<<<Categorical columns>>>>
 ['name', 'pid', 'upgrade_type_name', 'unit_type_name', 'ability_id', 'killer_pid']


In [11]:
print('<<<<Continuous columns>>>>\n',continuous_columns)

<<<<Continuous columns>>>>
 ['frame', 'second', 'count', 'ff_minerals_lost_army', 'ff_minerals_lost_economy', 'ff_minerals_lost_technology', 'ff_vespene_lost_army', 'ff_vespene_lost_economy', 'ff_vespene_lost_technology', 'food_made', 'food_used', 'minerals_collection_rate', 'minerals_current', 'minerals_killed', 'minerals_killed_army', 'minerals_killed_economy', 'minerals_killed_technology', 'minerals_lost', 'minerals_lost_army', 'minerals_lost_economy', 'minerals_lost_technology', 'minerals_used_active_forces', 'minerals_used_current', 'minerals_used_current_army', 'minerals_used_current_economy', 'minerals_used_current_technology', 'minerals_used_in_progress', 'minerals_used_in_progress_army', 'minerals_used_in_progress_economy', 'minerals_used_in_progress_technology', 'resources_killed', 'resources_lost', 'resources_used_current', 'resources_used_in_progress', 'vespene_collection_rate', 'vespene_current', 'vespene_killed', 'vespene_killed_army', 'vespene_killed_economy', 'vespene_k

There are 13 categorical columns and 55 continuous columns. 

1 of the continuous columns is 'count', which appears to count the number of an event that happen during a frame. This column will be used to insert the values upon dummification of the categorical columns. In other words, when categories are converted to dummy variables the value inside a column, for a specific row, will be multiplied by the value of the count column. In general, count appears to remain at 1, but this is done just in case it is not true for every replay. After the multiplication, the count column should be dropped.

### Continuous Columns
The value of the columns in the continuous set should be investigated to determine the best way of aggregating the values. For example, mineral collection rate would need to be averaged over the time chunks (this is stored as a value of Minerals collected per Minute), whereas the number of units killed would need to be summed. It would be best to include an aggregation type column in the `column_keep_df` which could be used during aggregation. This value can be added to the raw_columns_list.csv to automate the process.

In [12]:
event_df[continuous_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
frame,151759.0,20436.349119,11404.383827,0.0,10572.0,20413.0,30259.0,40406.0
second,151759.0,1276.807497,712.771601,0.0,660.0,1275.0,1891.0,2525.0
count,117.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
ff_minerals_lost_army,505.0,263.089109,482.600904,0.0,0.0,0.0,300.0,1400.0
ff_minerals_lost_economy,505.0,146.435644,167.044715,0.0,0.0,150.0,400.0,400.0
ff_minerals_lost_technology,505.0,64.752475,119.378301,0.0,0.0,0.0,100.0,400.0
ff_vespene_lost_army,505.0,131.336634,241.408823,0.0,0.0,0.0,150.0,700.0
ff_vespene_lost_economy,505.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
ff_vespene_lost_technology,505.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
food_made,505.0,183.641584,76.416207,14.0,152.0,212.0,230.0,313.0


### Categorical Columns

All categorical columns with an aggregation type specified will be dummified so that value is one or zero. During aggregation each time chunk will be aggregated by multiplying a columns value by count, and then summing a count of the occurrences within that chunk. 
Aggregation for categorical variables will follow these steps:
1. Dummy variables will be created for all necessary categorical columns.
2. Separate temporary dfs will be created for each players PID, and one for events with out PID
3. A helper column will be added to the temp dfs, calculated from frame number, to identify grouping of rows into time chunks (for 10 second frames, this would be chunks of `10*fps`).
4. Each column in the temp df will have pid added to its name if not NaN and then the dfs will be recombined to produced single df with 2 columns for every variable with a pid attached.
5. Continuous columns can be dropped from the df where PID = NaN as continuous variables only apply to PIDs
6. Perform aggregation on recombined df

In [13]:
event_df[categorical_columns].describe(include='all').T


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
name,151759,25,CameraEvent,118124,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pid,141778.0,NaN,NaN,NaN,6.3217,4.446597,0.0,2.0,6.0,10.0,14.0
upgrade_type_name,117,56,SprayZerg,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
unit_type_name,5676,108,Larva,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ability_id,8214.0,NaN,NaN,NaN,1131.562698,3104.606535,0.0,0.0,0.0,0.0,23136.0
killer_pid,2957.0,NaN,NaN,NaN,1.313155,0.463855,1.0,1.0,1.0,2.0,2.0


#### 1. Dummy variable creation
Before creating Dummy variables it makes sense to remove all rows that do not contain information about the PIDs that we are interested in (pid 1 and 2). Rows that include data about game information will have a PID of NaN, so we will keep these too.

Initially, the first dummy column created will be dropped. This may be revised later.


In [14]:
# remove all rows where pid not NaN and pid = 1 or 2 

event_df = event_df.loc[~(
    (event_df['pid'].notnull()) 
    & (event_df['pid'] != 1) 
    & (event_df['pid'] != 2))]

event_df.shape

(31727, 60)

In [15]:
event_df['pid'].isna().sum()

9981

In [16]:
# list of column_name in column_keep_df with agg_type not equal to none or
agg_columns = column_keep_df.loc[
    (column_keep_df['agg_type'] != 'none') 
    & (column_keep_df['agg_type'] != 'diff')
    & (column_keep_df['categorical_bool'] == 1),
    'column_name'].tolist()

# create dummies from event_df for all columns in agg_columns
event_df = pd.get_dummies(
    event_df, 
    prefix=agg_columns,
    columns=agg_columns,
    drop_first=True)

event_df



,frame,pid,second,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,...,unit_type_name_VespeneGeyser,unit_type_name_Viper,unit_type_name_VoidRay,unit_type_name_WarpGate,unit_type_name_WarpPrism,unit_type_name_WarpPrismPhasing,unit_type_name_Zealot,unit_type_name_Zergling,unit_type_name_ZerglingBurrowed,killer_pid_2.0
0,0,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,0,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,0,1.0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,0,1.0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,0,1.0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151751,40344,NaN,2521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
151752,40353,NaN,2522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
151756,40374,NaN,2523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
151757,40389,NaN,2524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


#### 2. Split into temporary dfs
3 temp dfs will be created, one for each PID and one for PID of NaN.

In [102]:
# df for PID = 1
df_1 = event_df.loc[
    event_df['pid'] == 1].copy()

# df for PID = 2
df_2 = event_df.loc[
    event_df['pid'] == 2].copy()

# df for PID is NaN
df_nan = event_df.loc[
    event_df['pid'].isnull()].copy()



#### 3. Add helper column to temp dfs
A helper column will be added that calculates which frame interval a row is in. This will be used with `groupby()` to aggregate the three dfs

In [81]:
# function to add helper columns for aggregation by second
def add_frame_aggregation_column(df, period, fps, new_column_name='agg_column'):
    """
    add_frame_aggregation_column 
    Adds a column to the dataframe that contains the second number at the start of the chunk. This second number will be used to aggregate the data.

    Args:
        df (DataFrame): DataFrame to add the column to
        period (int): The number of frames in a chunk
        fps (int): The number of frames per second
        new_column_name (str, optional): The name of the new column to add. Defaults to 'agg_column'.

    Returns:
        DataFrame: The dataframe with the new column added
    """    
    assert isinstance(df, pd.DataFrame), 'df must be a dataframe'

    # divide frame by period and round down to integer
    # then multiply by period / fps to get the start second of the chunk
    df[new_column_name] = (df['frame'] // period) * period / fps

    return df



In [103]:
fps = replay.game_fps
seconds_per_chunk = 10
period = int(fps * seconds_per_chunk)

df_1 = add_frame_aggregation_column(
                    df_1, period, fps, 'start_second').copy()
df_2 = add_frame_aggregation_column(
                    df_2, period, fps, 'start_second').copy()
df_nan = add_frame_aggregation_column(
                    df_nan, period, fps, 'start_second').copy()

In [83]:
# add pid identifier to all columns in df_1 and df_2
# drop frame, second, pid
def drop_my_columns(df, drop_columns=['frame', 'second', 'pid']):
    # drop columns inplace
    df.drop(drop_columns, axis=1, inplace=True)

    return df

def add_pid(df, pid):
    # rename all columns by adding 'pid_' to the front
    df.columns = ['pid_' + str(pid) + '_' + col for col in df.columns]

    return df


In [104]:
# loop through dfs and drop columns, 
# for i, df in enumerate([df_1, df_2]):
#     df = drop_my_columns(df).copy()
#     df.set_index('start_second', inplace=True)
#     df = add_pid(df, i + 1).copy()

df_1 = drop_my_columns(df_1).copy()
df_1.set_index('start_second', inplace=True)
df_1 = add_pid(df_1, 1).copy()

df_2 = drop_my_columns(df_2).copy()
df_2.set_index('start_second', inplace=True)
df_2 = add_pid(df_2, 2).copy()


In [85]:
df_nan

,frame,pid,second,count,ff_minerals_lost_army,ff_minerals_lost_economy,ff_minerals_lost_technology,ff_vespene_lost_army,ff_vespene_lost_economy,ff_vespene_lost_technology,...,unit_type_name_Viper,unit_type_name_VoidRay,unit_type_name_WarpGate,unit_type_name_WarpPrism,unit_type_name_WarpPrismPhasing,unit_type_name_Zealot,unit_type_name_Zergling,unit_type_name_ZerglingBurrowed,killer_pid_2.0,start_second
19,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
20,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
21,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
22,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
23,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151749,40339,NaN,2521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,2520.0
151751,40344,NaN,2521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,2520.0
151752,40353,NaN,2522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,2520.0
151756,40374,NaN,2523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,2520.0


In [86]:
# drop continuous_columns from df_nan
columns_to_drop = continuous_columns[:] + ['pid']
df_nan.drop(columns_to_drop, axis=1, inplace=True)
df_nan.set_index('start_second', inplace=True)

# drop all columns except those with sum > 0
df_nan = df_nan.loc[:,df_nan.sum(axis=0) > 0].copy()


In [88]:
df_1.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
count,69.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
ff_minerals_lost_army,252.0,526.388889,573.335157,0.0,0.0,300.0,900.0,1400.0
ff_minerals_lost_economy,252.0,74.404762,75.146887,0.0,0.0,0.0,150.0,150.0
ff_minerals_lost_technology,252.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
ff_vespene_lost_army,252.0,263.194444,286.667579,0.0,0.0,150.0,450.0,700.0
...,...,...,...,...,...,...,...,...
unit_type_name_Zealot,10567.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
unit_type_name_Zergling,10567.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
unit_type_name_ZerglingBurrowed,10567.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
killer_pid_2.0,10567.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [89]:
df_2.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
count,48.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
ff_minerals_lost_army,253.0,0.830040,4.353293,0.0,0.0,0.0,0.0,30.0
ff_minerals_lost_economy,253.0,218.181818,199.566630,0.0,0.0,400.0,400.0,400.0
ff_minerals_lost_technology,253.0,129.249012,141.891381,0.0,0.0,100.0,250.0,400.0
ff_vespene_lost_army,253.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
unit_type_name_Zealot,11179.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
unit_type_name_Zergling,11179.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
unit_type_name_ZerglingBurrowed,11179.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
killer_pid_2.0,11179.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [90]:
df_nan.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
name_UnitBornEvent,9981.0,0.360986,0.480311,0.0,0.0,0.0,1.0,1.0
name_UnitDiedEvent,9981.0,0.362489,0.480743,0.0,0.0,0.0,1.0,1.0
name_UnitDoneEvent,9981.0,0.047791,0.213334,0.0,0.0,0.0,0.0,1.0
name_UnitInitEvent,9981.0,0.049594,0.217116,0.0,0.0,0.0,0.0,1.0
name_UnitOwnerChangeEvent,9981.0,0.007114,0.084045,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
unit_type_name_WarpPrismPhasing,9981.0,0.000100,0.010010,0.0,0.0,0.0,0.0,1.0
unit_type_name_Zealot,9981.0,0.007815,0.088060,0.0,0.0,0.0,0.0,1.0
unit_type_name_Zergling,9981.0,0.036569,0.187712,0.0,0.0,0.0,0.0,1.0
unit_type_name_ZerglingBurrowed,9981.0,0.004308,0.065499,0.0,0.0,0.0,0.0,1.0
